<a href="https://colab.research.google.com/github/alessioborgi/DL_Project/blob/main/try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Load the MS MARCO dataset
ms_marco = load_dataset("microsoft/ms_marco", "v2.1", split="train")

# Display a sample
print(ms_marco[0])

train_data = ms_marco.shuffle(seed=42).select(range(80000))
validation_data = ms_marco.shuffle(seed=42).select(range(8000))

validation-00000-of-00001.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00000-of-00007.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

train-00001-of-00007.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

train-00002-of-00007.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

train-00003-of-00007.parquet:   0%|          | 0.00/242M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/204M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/101093 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/808731 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/101092 [00:00<?, ? examples/s]

{'answers': ['The immediate impact of the success of the manhattan project was the only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.'], 'passages': {'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'passage_text': ['The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.', 'The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.', 'Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this projec

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from transformers import AutoModel, AutoTokenizer

# Load a pre-trained model for embeddings
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Generate embeddings for documents
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyError: 'passage'

In [ ]:
for doc in train_data:
    print(doc)
    break

{'answers': ['Krishnaraj Rai'], 'passages': {'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'passage_text': ["Pics/Satej Shinde, Shadab Khan, Bipin Kokate. Aishwarya Rai Bachchan's father Krishnaraj Rai passed away on Saturday at a suburban hospital here due to health issues. He was hospitalised a few weeks ago and was later shifted to the ICU.", "Aishwarya's father Krishnaraj Rai's funeral held, celebs pay last respects. Actress Aishwarya Rai Bachchan 's father Krishnaraj Rai passed away recently at a suburban hospital in Mumbai. Rai was hospitalised a few weeks ago and later shifted to the ICU. The funeral took place at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Bachchan also offered prayers.", "Aishwarya's father Krishnaraj Rai passed away at the Lilavati Hospital in suburban Bandra, the actress' publicist stated. The funeral took place around 8.30 pm at Vile Parle Seva Sansthan Shamshan Bhoomi where Aishwarya's father-in-law Amitabh Bachchan al

In [ ]:
# Create embeddings
embeddings = [embed_text(doc['passage_text']) for doc in train_data]

# Apply clustering
clustering = AgglomerativeClustering(n_clusters=100)
labels = clustering.fit_predict(embeddings)

# Assign semantically structured identifiers
ssids = [f"Cluster_{label}" for label in labels]
train_data = [{'text': doc['passage_text'], 'docid': ssid} for doc, ssid in zip(train_data, ssids)]

In [ ]:
from torch.utils.data import Dataset

class DSIInputs2TargetDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        doc_text = self.data[idx]['text']
        docid = self.data[idx]['docid']

        # Tokenize input (document text)
        source = self.tokenizer(
            doc_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Tokenize target (structured identifier)
        target = self.tokenizer(
            docid,
            max_length=10,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': source['input_ids'].squeeze(),
            'attention_mask': source['attention_mask'].squeeze(),
            'labels': target['input_ids'].squeeze()
        }


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

# Initialize model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Create the dataset
train_dataset = DSIInputs2TargetDataset(train_data, tokenizer)
eval_dataset = DSIInputs2TargetDataset(validation_data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./dsi_output",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(model, dataset):
    model.eval()
    hits_at_1 = 0
    total = len(dataset)

    for data in dataset:
        inputs = tokenizer(data['text'], return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model.generate(**inputs)
        predicted_docid = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if predicted_docid == data['docid']:
            hits_at_1 += 1

    accuracy = hits_at_1 / total
    print(f"Hits@1 Accuracy: {accuracy * 100:.2f}%")

# Run evaluation
evaluate(model, eval_dataset)
